<a href="https://colab.research.google.com/github/linyuehzzz/census_privacy/blob/main/topdown_ols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


**Budget allocation**

In [4]:
scale = 3   # county:0, tract:1, bg:2, block:3
if scale == 0:
    f1, f2, f3, f4, f6, f7, f8, f9, f10, f11 = 1, 447/4099*10/4097, 447/4099*10/4097, 447/4099*10/4097, 447/4099*10/4097, 447/4099*28/4097, 447/4099*28/4097, 447/4099*10/4097, 447/4099*101/4097, 447/4099*754/4097
elif scale == 1:
    f1, f2, f3, f4, f6, f7, f8, f9, f10, f11 = 687/4099*1567/4102, 687/4099*4/2051, 687/4099*5/4102, 687/4099*5/4102, 687/4099*5/4102, 687/4099*1933/4102, 687/4099*10/2051, 687/4099*5/4102, 687/4099*67/4102, 687/4099*241/2051
elif scale == 2:
    f1, f2, f3, f4, f6, f7, f8, f9, f10, f11 = 1256/4099*1705/4099, 1256/4099*3/4099, 1256/4099*3/4099, 1256/4099*3/4099, 1256/4099*3/4099, 1256/4099*1055/4099, 1256/4099*9/4099, 1256/4099*3/4099, 1256/4099*24/4099, 1256/4099*1288/4099
else:
    f1, f2, f3, f4, f6, f7, f8, f9, f10, f11 = 165/4099*5/4097, 165/4099*9/4097, 165/4099*5/4097, 165/4099*5/4097, 165/4099*5/4097, 165/4099*21/4097, 165/4099*21/4097, 165/4099*5/4097, 165/4099*71/4097, 165/4099*3945/4097

**Define strategy matrix and rescale (for homoskedastic errors)**

In [5]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
import torch
import numpy as np
import math
import lib.cdp2adp as cdp2adp

# setup parameters
eps = 17.14
delta = 1e-10
# rho = 1
rho = cdp2adp.cdp_rho(eps, delta) 
n1, n2, n3, n4 = 8, 2, 2, 63        # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)
N = n1 * n2 * n3 * n4         # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)
q1, q2, q3, q4, q6, q7, q8, q9, q10, q11 = [], [], [], [], [], [], [], [], [], []

AA = torch.tensor(range(N))
AA = AA.reshape([n1, n2, n3, n4])

## Q1: TOTAL (1 cell)
q1 = np.full((1, N), rho * f1)

## Q2: CENRACE (63 cells)
q2 = np.zeros((n4, N))
for x in range(n4):     # race
    hist_idx = torch.flatten(AA[:, :, :, x]).tolist()
    for i in hist_idx:
        q2[x, i] = math.sqrt(rho * f2)

## Q3: HISPANIC (2 cells)
q3 = np.zeros((n3, N))
for x in range(n3):
    hist_idx = torch.flatten(AA[:, :, x, :]).tolist()
    for i in hist_idx:
        q3[x, i] = math.sqrt(rho * f3)   

## Q4: VOTINGAGE (2 cells)
q4 = np.zeros((n2, N))
for x in range(n2):
    hist_idx = torch.flatten(AA[:, x, :, :]).tolist()
    for i in hist_idx:
        q4[x, i] = math.sqrt(rho * f4) 

## Q6: HHGQ (8 cells)
q6 = np.zeros((n1, N))
for x in range(n1):
    hist_idx = torch.flatten(AA[x, :, :, :]).tolist()
    for i in hist_idx:
        q6[x, i] = math.sqrt(rho * f6) 

## Q7: HISPANIC*CENRACE (126 cells)
q7 = np.zeros((n3*n4, N))
row_idx = 0 
for x in range(n3): 
    for y in range(n4):
        hist_idx = torch.flatten(AA[:, :, x, y]).tolist()
        for i in hist_idx:
            q7[row_idx, i] = math.sqrt(rho * f7)
        row_idx += 1

## Q8: VOTINGAGE*CENRACE (126 cells)
q8 = np.zeros((n2*n4, N))
row_idx = 0 
for x in range(n2): 
    for y in range(n4):
        hist_idx = torch.flatten(AA[:, x, :, y]).tolist()
        for i in hist_idx:
            q8[row_idx, i] = math.sqrt(rho * f8)
        row_idx += 1

## Q9: VOTINGAGE*HISPANIC (4 cells)
q9 = np.zeros((n2*n3, N))
row_idx = 0 
for x in range(n2): 
    for y in range(n3):
        hist_idx = torch.flatten(AA[:, x, y, :]).tolist()
        for i in hist_idx:
            q9[row_idx, i] = math.sqrt(rho * f9)
        row_idx += 1

## Q10: VOTINGAGE*HISPANIC*CENRACE (252 cells)
q10 = np.zeros((n2*n3*n4, N))
row_idx = 0 
for x in range(n2):
    for y in range(n3):
        for z in range(n4):
            hist_idx = torch.flatten(AA[:, x, y, z]).tolist()
            for i in hist_idx:
                q10[row_idx, i] = math.sqrt(rho * f10)
            row_idx += 1

## Q11: HHGQ*VOTINGAGE*HISPANIC*CENRACE (2,016 cells)
q11 = np.zeros((N, N))
np.fill_diagonal(q11, math.sqrt(rho * f11))

## stack all queries -> strategy matrix
A = np.vstack((q1, q2, q3, q4, q6, q7, q8, q9, q10, q11))

print(q1.shape, q2.shape, q3.shape, q4.shape, q6.shape, q7.shape, q8.shape, q9.shape, q10.shape, q11.shape, A.shape)
A

/content/gdrive/My Drive/Colab Notebooks/census_privacy
(1, 2016) (63, 2016) (2, 2016) (2, 2016) (8, 2016) (126, 2016) (126, 2016) (4, 2016) (252, 2016) (2016, 2016) (2600, 2016)


array([[1.25532154e-04, 1.25532154e-04, 1.25532154e-04, ...,
        1.25532154e-04, 1.25532154e-04, 1.25532154e-04],
       [1.50318953e-02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.50318953e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.14713949e-01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.14713949e-01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 3.14713949e-01]])

**Define workload query**

Detailed

In [22]:
import numpy as np

# setup parameters
n1, n2, n3, n4 = 8, 2, 2, 63        # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)
N = n1 * n2 * n3 * n4         # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)

W = np.zeros((N, N))
np.fill_diagonal(W, 1)
W.shape

(2016, 2016)

Race * Voting age

In [1]:
import numpy as np
import torch

# setup parameters
n1, n2, n3, n4 = 8, 2, 2, 63        # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)
N = n1 * n2 * n3 * n4         # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)

AA = torch.tensor(range(N))
AA = AA.reshape([n1, n2, n3, n4])

W = np.zeros((n2 * n4, N))
row_idx = 0
for x in range (n2):
    for y in range(n4):
        hist_idx = torch.flatten(AA[:, x, :, y]).tolist()
        for i in hist_idx:
            W[row_idx, i] = 1
        row_idx += 1

Race

In [20]:
import numpy as np
import torch

# setup parameters
n1, n2, n3, n4 = 8, 2, 2, 63        # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)
N = n1 * n2 * n3 * n4         # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)

AA = torch.tensor(range(N))
AA = AA.reshape([n1, n2, n3, n4])

t = 7
W = np.zeros((t, N))
col_two_or_more_races = []
for x in range(n4):
    if x >= 0 and x <= 5:     
        hist_idx = torch.flatten(AA[:, :, :, x]).tolist()
        for i in hist_idx:
            W[x, i] = 1
    else:
        hist_idx = torch.flatten(AA[:, :, :, x]).tolist()
        col_two_or_more_races.extend(hist_idx)
for i in hist_idx:
    W[6, i] = 1
W.shape

(7, 2016)

OLS variance matrix

$$Var(W\hat{x})=W(\tilde{A}^T\tilde{A})^{-1}W^T$$

In [6]:
import numpy as np

var = W @ np.linalg.inv(A.T @ A) @ W.T
var[0, 0]

122.63620215995955